# Convert TF Object Detection Models for Open Vino Inference Engine

## configuration

In [ ]:
import tensorflow as tf
import os

workspace_dir = '/home/manuel/Bachelor_Arbeit/workspace'
dataset = 'OI_Animals_9'
model = 'faster_rcnn_resnet50_coco_2018_01_28_out'

checkpoint = 199666
model_type_json = 'faster_api'
pipeline_config_file = 'faster_res50'
img_size = 600
channel = 3
batch_size = 1

model_type = {
    'faster': '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/faster_rcnn_support.json',
    'faster_api': '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/faster_rcnn_support_api_v1.14.json',
    'ssd' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_support.json',
    'ssd_api' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_support_api_v1.14.json',
    'rfcn' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/rfcn_support.json',
    'rfcn_api' : '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/rfcn_support_api_v1.14.json'
}

config_file = {
    'ssd_mob': 'ssd_mobilenet_v2_coco.config', 
    'ssd_mob_oi': 'ssd_mobilenet_v2_oid_v4.config', 
    'ssd_inc':  'ssd_inception_v2_coco.config',
    'faster_inc': 'faster_rcnn_inception_v2_coco.config',
    'rfcn_res101':  'rfcn_resnet101_pets.config',
    'faster_res50': 'faster_rcnn_resnet50_coco.config'
}

pb_file = os.path.join(workspace_dir, dataset, model, 'exported', 'frozen_inference_graph.pb')

configuration_file = model_type[model_type_json]

pipeline = os.path.join(workspace_dir, dataset, model, config_file[pipeline_config_file])

plugin_device = 'MYRIAD'

data_type = 'FP16'

input_shape_str = '['+ str(batch_size) +',' + str(img_size) + ',' + str(img_size) + ',' + str(channel) + ']'

output_dir = os.path.join(workspace_dir, dataset, model, 'open_vino')

mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'
input_shape_str

## export frozen graph

In [ ]:
checkpoint_str = os.path.join(workspace_dir, dataset, model, 'model.ckpt-' + str(checkpoint))
exported = os.path.join(workspace_dir, dataset, model, 'exported')

!python3 /home/manuel/Bachelor_Arbeit/TensorFlow_Models/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path $pipeline \
    --trained_checkpoint_prefix $checkpoint_str \
    --output_directory $exported

##  convert for open vino

In [ ]:

!python3 {mo_tf_path} \
--input_model {pb_file} \
--tensorflow_use_custom_operations_config {configuration_file} \
--input_shape {input_shape_str} \
--data_type {data_type} \
--input image_tensor \
--tensorflow_object_detection_api_pipeline_config {pipeline} \
--output_dir {output_dir}


## Inference

In [ ]:
from openvino.inference_engine import IENetwork, IEPlugin, IECore
import cv2
import numpy as np
import time
import os
from random import shuffle
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
workspace_dir = '/home/manuel/Bachelor_Arbeit/workspace'
dataset = 'OI_Animals_Augmented_9_2000'
model = 'ssd_inception_v2_coco_2018_01_28_out'

In [ ]:
model_xml = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.xml')
model_bin = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.bin')

labels = [l.strip() for l in open(os.path.join(workspace_dir, dataset, 'classes.txt')).readlines()]

test_images_dir = os.path.join(workspace_dir, dataset, 'validation')

assert os.path.isfile(model_bin)
assert os.path.isfile(model_xml)
assert os.path.isdir(test_images_dir)

In [ ]:
# collect images
if os.path.isdir(test_images_dir):
    images = [
        os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img[-3:] == 'jpg']
    print(str(len(images)) + ' images found')
    shuffle(images)
elif os.path.isfile(test_images_dir) and test_images_dir[-3:] == 'jpg':
    print('one image found')
    images = [test_images_dir]
else:
    print('wrong input')
    exit()
    
frame = cv2.imread(images[0])
initial_h, initial_w = frame.shape[: 2]

In [ ]:
# create the infer net
ie = IECore()
net = IENetwork(model=model_xml, weights=model_bin)

exec_net = ie.load_network(network=net, num_requests=2, device_name='MYRIAD')

img_info_input_blob = None
feed_dict = {}
for blob_name in net.inputs:
    if len(net.inputs[blob_name].shape) == 4:
        input_blob = blob_name
    elif len(net.inputs[blob_name].shape) == 2:
        img_info_input_blob = blob_name
    else:
        raise RuntimeError("Unsupported {}D input layer '{}'. Only 2D and 4D input layers are supported"
                               .format(len(net.inputs[blob_name].shape), blob_name))

assert len(net.outputs) == 1, "Demo supports only single output topologies"

out_blob = next(iter(net.outputs))
n, c, h, w = net.inputs[input_blob].shape
if img_info_input_blob:
    feed_dict[img_info_input_blob] = [h, w, 1]

cur_request_id = 0
next_request_id = 1

frame = cv2.imread(images[0])
initial_h, initial_w = frame.shape[: 2]

In [ ]:
# inference
# not yet implemented